In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from torch.optim.optimizer import Optimizer
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
from transformers import pipeline
import warnings 
warnings.filterwarnings('ignore')
from pytorch_lightning import seed_everything
from torch.utils.data import DataLoader
import os
import gc
gc.collect()

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [15]:
def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    seed_everything(seed)
    return random_state
random_state = set_seed(42)

Global seed set to 42


In [16]:
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load('Product_Data_SQuAD_model_2144.pt'))
model.eval()
nlp = pipeline('question-answering', model=model.to('cpu'), tokenizer=tokenizer)
gc.collect()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

21

In [17]:
def model_predict(nlp,df):
    table = pd.DataFrame()
    idx_list = sorted(df.index.tolist())
    for i in tqdm(idx_list):
        sample = df.loc[[i]]
        string_X_train = sample['string_X_train'].values[0]
        QA_input = {
            'question': 'What is the product name?',
            'context': string_X_train
        }
        res = nlp(QA_input)
        predict = QA_input['context'][res['start']:res['end']]
        row = pd.DataFrame({'predict:':predict},index=[i])
        table = table.append(row)
    return table

In [18]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm


df = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')
display(df.head(3))
display(df[df['品名']=='MA'])

產品集合 = set(df['品名'].values)

'''
train_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)
train_df = train_df.dropna(axis=0)
display(train_df.head(3))

val_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)
val_df = val_df.dropna(axis=0)
display(val_df.head(3))
'''

,RIGID,分機,公司代號,公司事業部門,品名,新增
0,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID FLAKE PEARLS MICROPEARLS,NaN
1,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID,NaN
2,黃淑玲,7190,11,台塑塑膠部,COMMODITY FORMOSACN,NaN


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


"\ntrain_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)\ntrain_df = train_df.dropna(axis=0)\ndisplay(train_df.head(3))\n\nval_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)\nval_df = val_df.dropna(axis=0)\ndisplay(val_df.head(3))\n"

In [19]:
品名2部門 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號 = dict(zip(df['品名'],df['公司代號']))

In [20]:
val_df = pd.read_csv('preprocess_for_SQUAD_product.csv',index_col=0)[['45A','Y_label','EXPNO']]
print(val_df.shape)
val_df.columns = ['string_X_train','Y_label','EXPNO']
val_df

(3224, 3)


,string_X_train,Y_label,EXPNO
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18
...,...,...,...
3613,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A
3614,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41
3615,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A
3616,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A


In [21]:
train_df = val_df

# 如果品名是單詞的話 前後加個空白

In [22]:
for i in df.index:
    if ' ' not in df.loc[i,'品名']:
        name = df.loc[i,'品名']
        df.loc[i,'品名'] = f' {name} '
        assert df.loc[i,'品名'][0] == ' '
        assert df.loc[i,'品名'][-1] == ' '

In [23]:
display(df[df['品名']=='MA'])
display(df[df['品名']==' MA '])

,RIGID,分機,公司代號,公司事業部門,品名,新增


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


In [24]:
產品集合 = set(df['品名'].values)

# find_fail_sample and drop fail_sample

In [25]:
def find_fail_sample(df):
    fails = []
    for i in df.index:
        context = df.loc[i,'string_X_train']
        answer = df.loc[i,'Y_label']
        if str(answer) not in str(context):
            fails.append(i)
    return fails
train_fails = find_fail_sample(train_df)
val_fails = find_fail_sample(val_df)
print(train_fails,val_fails)
display(val_df.loc[val_fails])
print(val_df.shape)
val_df = val_df.drop(val_fails,axis=0)
print(val_df.shape)

[182, 489, 522, 746, 749, 759, 760, 761, 830, 833, 834, 876, 877, 892, 909, 921, 924, 926, 958, 988, 999, 1004, 1005, 1008, 1010, 1047, 1049, 1084, 1133, 1136, 1137, 1155, 1204, 1210, 1267, 1302, 1317, 1319, 1328, 1370, 1400, 1483, 1503, 1504, 1514, 1554, 1555, 1591, 1604, 1646, 1653, 1680, 1682, 1683, 1716, 1717, 1719, 1721, 1724, 1757, 1758, 1783, 1834, 1874, 1875, 1965, 1967, 1968, 2070, 2211, 2213, 2221, 2232, 2236, 2249, 2285, 2327, 2357, 2391, 2393, 2397, 2482, 2489, 2506, 2545, 2557, 2577, 2578, 2600, 2604, 2612, 2684, 2713, 2724, 2772, 2830, 2832, 2837, 2842, 2843, 2844, 2845, 2848, 2851, 2916, 2966, 2989, 2990, 3004, 3028, 3030, 3087, 3088, 3137, 3166, 3170, 3198, 3219, 3220, 3229, 3239, 3252, 3299, 3300, 3343, 3397, 3398, 3403, 3558, 3588, 3591, 3594, 3595, 3599, 3609, 3612] [182, 489, 522, 746, 749, 759, 760, 761, 830, 833, 834, 876, 877, 892, 909, 921, 924, 926, 958, 988, 999, 1004, 1005, 1008, 1010, 1047, 1049, 1084, 1133, 1136, 1137, 1155, 1204, 1210, 1267, 1302, 1317, 13

,string_X_train,Y_label,EXPNO
182,PHTHALIC ANHYDRIDE 504 MT AT USD 1010 MT x 000...,NaN,27
489,108 MT PHTHALIC ANHYDRIDE x 000 Dx 000 DCI F M...,NaN,27
522,PHTHALIC ANHYDRIDE 504 MT AT USD 980 MT x 000 ...,NaN,27
746,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN,27
749,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN,27
...,...,...,...
3594,3600 MT 2 F CL x 000 D PHTHALIC ANHYDRIDE x 00...,NaN,27
3595,PHTHALIC ANHYDRIDE PAx 000 D QT Y 54 MTS x 000...,NaN,27
3599,108 MT 6 F CL OF PHTHALIC ANHYDRIDE AT 98000 P...,NaN,27
3609,108 MT 6 F CL OF PHTHALIC ANHYDRIDE AT USD 960...,NaN,27


(3224, 3)
(3088, 3)


In [26]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if p in df.loc[i,'string_X_train']:
                products.append(p)
        labels[i] = products
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = labels.values()
    return predict
predict = Collection_method(val_df,產品集合)
result = val_df.join(predict)

  0%|          | 0/3088 [00:00<?, ?it/s]

In [27]:
result

,string_X_train,Y_label,EXPNO,predict
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11,"[PVC RESIN B-57, RESIN , PVC RESIN]"
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27,"[PHTHALIC ANHYDRIDE , PA , PHTHALIC ANHYDRIDE..."
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18,"[LLDPE TAISOX, TAISOX ]"
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18,"[HDPE TAISOX, HDPE TAISOX 8010, HDPE , TAISOX ]"
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18,"[HDPE TAISOX, HDPE TAISOX 8010, HDPE , TAISOX ]"
...,...,...,...,...
3613,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A,"[MONO ETHYLENE GLYCOL, ETHYLENE , INA ]"
3614,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41,"[VISCOSE RAYON STAPLE FIBER, RAYON STAPLE FIBE..."
3615,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]"
3616,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]"


In [28]:
not_find = []
for j,i in enumerate(result.iloc[:,3].values):
    if len(i) == 0:
        not_find.append(j)
len(not_find)

287

In [29]:
not_find_df = result.iloc[not_find]
not_find_df

,string_X_train,Y_label,EXPNO,predict
51,PVC SUSPENSION S65 - 112 MT AT USD 133000 PER ...,PE,11,[]
125,x 000 DP OM FORMOCON FM 130 18000 KG NET x 000...,PACK,1P,[]
203,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[]
204,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[]
217,x 000 DP OM FORMOCON FM 090 18000 KG NET x 000...,PACK,1P,[]
...,...,...,...,...
3560,ABS AG 15 AA - H x 000 D Q U A N T I T Y 18000...,ABS,4A,[]
3570,ABS AG 15 AA - H x 000 D Q U A N T I T Y 25200...,ABS,4A,[]
3600,EPICHLOROHYDRIN x 000 D QT Y 18400 x 000 DAS P...,EPICHLOROHYDRIN,1A,[]
3607,ACETONITRILE 128 MT x 000 DAT USD 3650 PER MT ...,ACETONITRILE,1A,[]


In [30]:
bert_predict = model_predict(nlp,not_find_df)
bert_predict

  0%|          | 0/287 [00:00<?, ?it/s]

,predict:
51,PVC SUSPENSION S65
125,PACKAGING
203,DOP
204,DOP
217,PACKAGING
...,...
3560,ABS
3570,ABS
3600,EPICHLOROHYDRIN
3607,ACETONITRILE


In [31]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [32]:
result.iloc[not_find,3] = [ [str(i)] for i in bert_predict['predict:'].values]
result.iloc[not_find]

,string_X_train,Y_label,EXPNO,predict
51,PVC SUSPENSION S65 - 112 MT AT USD 133000 PER ...,PE,11,[PVC SUSPENSION S65]
125,x 000 DP OM FORMOCON FM 130 18000 KG NET x 000...,PACK,1P,[PACKAGING]
203,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[DOP]
204,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[DOP]
217,x 000 DP OM FORMOCON FM 090 18000 KG NET x 000...,PACK,1P,[PACKAGING]
...,...,...,...,...
3560,ABS AG 15 AA - H x 000 D Q U A N T I T Y 18000...,ABS,4A,[ABS]
3570,ABS AG 15 AA - H x 000 D Q U A N T I T Y 25200...,ABS,4A,[ABS]
3600,EPICHLOROHYDRIN x 000 D QT Y 18400 x 000 DAS P...,EPICHLOROHYDRIN,1A,[EPICHLOROHYDRIN]
3607,ACETONITRILE 128 MT x 000 DAT USD 3650 PER MT ...,ACETONITRILE,1A,[ACETONITRILE]


In [33]:
get_jaccard_sim('MASS PVC RESIN B-57','PVC RESIN B-57')

0.75

In [34]:
for j in result.loc[51,'predict']:
    print(j)

PVC SUSPENSION S65


In [53]:
def get_acc(df,t=0.75):
    correct = []
    correct_label = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        if max(jacs) >= t:
            correct.append('yes')
        else:
            correct.append('no')
    return result['correct'].value_counts()['yes']/len(result)

In [54]:
def get_jac(df):
    all_jacs = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        all_jacs.append(max(jacs))
    return np.sum(all_jacs)/len(all_jacs)

# 表現

In [59]:
get_acc(result,t=1),get_acc(result,t=0.75),get_jac(result)

(0.8597797927461139, 0.8597797927461139, 0.8804388724401677)

In [60]:
部門_lst = []
for p_lst in tqdm(result['predict'].values):
    p = max(p_lst,key=len)
    jac_dict = {}
    for i in 品名2代號.keys():
        jac_dict[i] = get_jaccard_sim(i,p)
    部門_lst.append(品名2代號[max(jac_dict, key=jac_dict.get)])

  0%|          | 0/3088 [00:00<?, ?it/s]

In [61]:
result['預測部門代號'] = 部門_lst

In [62]:
result

,string_X_train,Y_label,EXPNO,predict,correct,預測部門代號
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11,"[PVC RESIN B-57, RESIN , PVC RESIN]",yes,11
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27,"[PHTHALIC ANHYDRIDE , PA , PHTHALIC ANHYDRIDE...",yes,27
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18,"[LLDPE TAISOX, TAISOX ]",yes,18
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18,"[HDPE TAISOX, HDPE TAISOX 8010, HDPE , TAISOX ]",yes,18
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18,"[HDPE TAISOX, HDPE TAISOX 8010, HDPE , TAISOX ]",yes,18
...,...,...,...,...,...,...
3613,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A,"[MONO ETHYLENE GLYCOL, ETHYLENE , INA ]",yes,2A
3614,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41,"[VISCOSE RAYON STAPLE FIBER, RAYON STAPLE FIBE...",yes,41
3615,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A
3616,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A


In [63]:
result['EXPNO'] = [ str(i).strip() for i in result['EXPNO']]
result['預測部門代號'] = [ str(i).strip() for i in result['預測部門代號']]

In [64]:
result[result['EXPNO']==result['預測部門代號']]

,string_X_train,Y_label,EXPNO,predict,correct,預測部門代號
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11,"[PVC RESIN B-57, RESIN , PVC RESIN]",yes,11
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27,"[PHTHALIC ANHYDRIDE , PA , PHTHALIC ANHYDRIDE...",yes,27
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18,"[LLDPE TAISOX, TAISOX ]",yes,18
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18,"[HDPE TAISOX, HDPE TAISOX 8010, HDPE , TAISOX ]",yes,18
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,18,"[HDPE TAISOX, HDPE TAISOX 8010, HDPE , TAISOX ]",yes,18
...,...,...,...,...,...,...
3613,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A,"[MONO ETHYLENE GLYCOL, ETHYLENE , INA ]",yes,2A
3614,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41,"[VISCOSE RAYON STAPLE FIBER, RAYON STAPLE FIBE...",yes,41
3615,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A
3616,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A


In [65]:
result[result['EXPNO']!=result['預測部門代號']]

,string_X_train,Y_label,EXPNO,predict,correct,預測部門代號
9,300000 BBL S - 10 P C T OF GASOIL 10 PPM UNIT ...,GASOIL,61,[ GASOIL ],yes,60
10,300000 BBL S - 10 P C T OF GASOIL 10 PPM UNIT ...,GASOIL,61,[ GASOIL ],yes,60
13,300000 BBL S - 10 P C T OF GASOIL 10 PPM UNIT ...,GASOIL,61,[ GASOIL ],yes,60
14,PRODUCT TETRAHYDROFURAN 998 PCT MIN QUANTITY 1...,TETRAHYDROFURAN,2P,[ TETRAHYDROFURAN ],yes,24
26,COMMODITY MONOETHYLENE GLYCOL MEG FIBER GRADE ...,MONOETHYLENE GLYCOL MEG FIBER GRADE,2A,[INA ],no,27
...,...,...,...,...,...,...
3544,QUANTITY 36 MTS OF POM FORMOCON AS FOLLOWS FM0...,PE,1P,[PER],no,11
3545,QUANTITY 36 MTS OF POM FORMOCON AS FOLLOWS FM0...,PE,1P,[PER],no,11
3546,QUANTITY 36 MTS OF POM FORMOCON AS FOLLOWS FM0...,PE,1P,[PER],no,11
3593,COMMODITY MONOETHYLENE GLYCOL x 000 D QUANTITY...,ETHYLENE,2A,[INA ],no,27


In [66]:
品名2代號['GASOIL'] # EXPNO是61 寶典是60

60

In [67]:
len(result[result['EXPNO']==result['預測部門代號']])/len(result)

0.7807642487046632

In [68]:
result.to_csv('submit_0726.csv')